In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('Groceries_dataset.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [4]:
data.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


### We need the data to be in the transactions format where each member's single purchase should contain all the items bought as a single row, so we process the dataframe to get into that format. 

In [5]:
import matplotlib.pyplot as plt

In [6]:
obj=data.groupby(by=['Member_number','Date'])

In [7]:
sam=obj.get_group((2552,'05-01-2015'))

In [8]:
sam['itemDescription'].values

array(['whole milk', 'tropical fruit', 'chocolate'], dtype=object)

In [9]:
data['itemDescription'].nunique()

167

In [10]:
transaction={}

In [11]:
for i,j in obj.groups.items():
    sam=obj.get_group(i)
    transaction[i]=sam['itemDescription'].values

In [12]:
C1=data['itemDescription'].value_counts()

In [13]:
C1

itemDescription
whole milk               2502
other vegetables         1898
rolls/buns               1716
soda                     1514
yogurt                   1334
                         ... 
rubbing alcohol             5
bags                        4
baby cosmetics              3
kitchen utensil             1
preservation products       1
Name: count, Length: 167, dtype: int64

In [14]:
c_list=sorted(C1.index)
C1=C1[c_list]

In [15]:
C1

itemDescription
Instant food products      60
UHT-milk                  323
abrasive cleaner           22
artif. sweetener           29
baby cosmetics              3
                         ... 
white bread               362
white wine                176
whole milk               2502
yogurt                   1334
zwieback                   60
Name: count, Length: 167, dtype: int64

In [16]:
import numpy as np
from scipy import stats as st

In [17]:
print('median:',np.median(C1),'\nmode:',st.mode(C1),'\nmean:',np.mean(C1))

median: 85.0 
mode: ModeResult(mode=array([5], dtype=int64), count=array([4])) 
mean: 232.125748502994


C:\Users\WELCOME\AppData\Local\Temp\ipykernel_1376\1486841782.py:1: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  print('median:',np.median(C1),'\nmode:',st.mode(C1),'\nmean:',np.mean(C1))


In [18]:
min_support_count=25

In [19]:
print(C1[C1>min_support_count])

itemDescription
Instant food products      60
UHT-milk                  323
artif. sweetener           29
baking powder             122
beef                      516
                         ... 
white bread               362
white wine                176
whole milk               2502
yogurt                   1334
zwieback                   60
Name: count, Length: 132, dtype: int64


In [20]:
import threading
import multiprocessing
import math
from itertools import combinations

In [21]:
class apriori:
    def __init__(self,transaction,min_count):
        self.transaction_set=transaction
        self.min_sup_count=min_count
        
    def c(self,C,C_count,i,n):
        for j in range(i,n):
            cn=0
            for k in self.transaction_set.values():
                if (all([item in k for item in C[j]])):
                    cn+=1
            C_count[C[k]]=cn
            
        return C_count
    
    def count_val(self,C):
        C_count=pd.Series([1 for i in range(len(C))])
        C_count.index=C
        '''if(math.ceil(len(C)/10)>10):
            for i in range(0,len(C),math.ceil(len(C)/10)):
                Dynamic_Thread_Name=f't{i}'
                globals()[Dynamic_Thread_Name]=threading.Thread(target=self.c,args=(C,C_count,i,math.ceil(len(C)/10)+i if (math.ceil(len(C)/10)+i)<len(C) else len(C)))
                globals()[Dynamic_Thread_Name].start()
                globals()[Dynamic_Thread_Name].join()
            
            t1=threading.Thread(target=self.c,args=(C,C_count,len(C)-math.floor(len(C)%math.ceil(len(C)/10)),len(C)))
            t1.start()
            t1.join()
            
        else:'''
        for i in range(len(C)):
            cn=0
            for j in self.transaction_set.values():
                if (all([item in j for item in C[i]])):
                    cn+=1
            C_count[C[i]]=cn
            
        return C_count
    
    def Candidate(self,k):
        att=f'L{k-1}'
        L_prev=getattr(self,att,None)
        #join step
        C=list()
        ind=L_prev.index
        for i in range(len(ind)):
            for j in range(i+1,len(ind)):
                list1=ind[i]
                list2=ind[j]
                if(all([list1[a]==list2[a] for a in range(len(list1)-1)])):
                    C.append(tuple([a for a in list1[:-1]]+[list1[-1]]+[list2[-1]]))
                    
        print(C)
        
        #prune step
        for i in C:
            n_count=0
            sam=combinations(i,k-1)
            sam=list(sam)
            for item in sam:
                if item in ind:
                    n_count+=1
                    
            if(n_count!=k):
                C.remove(i)
                
        print(C)
            
            
        C=self.count_val(C)
        print(C)
        L=C[C>=self.min_sup_count]
        if(L.empty):
            return 0
        elif(len(L)==0):
            return 0
        else:
            att=f'L{k}'
            setattr(self,att,L)
            return 1
        
    
    def disp(self,k):
        
        for i in range(1,k):
            print(f'L{i}:',getattr(self,f'L{i}',None))
    
    def define(self):
        C1=[]
        for i in self.transaction_set.values():
            for j in i:
                C1.append(j)
                
        C1=pd.Series(sorted(C1)).value_counts()
        temp=sorted(C1.index)
        C1=C1[temp]
        L1=C1[C1>=self.min_sup_count]
        setattr(self,'L1',L1)
        ind=self.L1.index
        C2=[]
        for i in range(len(ind)):
            for j in range(i+1,len(ind)):
                list1=ind[i]
                list2=ind[j]
                C2.append(tuple([list1,list2])) 
        C2=self.count_val(C2)
        L2=C2[C2>=self.min_sup_count]
        setattr(self,'L2',L2)
        k=3
        flag=1
        while(flag==1):
            flag=self.Candidate(k)
            k+=1
            print('k:',k,'L:',getattr(self,f'L{k-1}',None))
            
        
        self.disp(k-1)
    

In [22]:
a=apriori(transaction,min_support_count)

In [23]:
a.define()

[('UHT-milk', 'other vegetables', 'rolls/buns'), ('UHT-milk', 'other vegetables', 'whole milk'), ('UHT-milk', 'rolls/buns', 'whole milk'), ('beef', 'citrus fruit', 'newspapers'), ('beef', 'citrus fruit', 'other vegetables'), ('beef', 'citrus fruit', 'root vegetables'), ('beef', 'citrus fruit', 'soda'), ('beef', 'citrus fruit', 'whole milk'), ('beef', 'citrus fruit', 'yogurt'), ('beef', 'newspapers', 'other vegetables'), ('beef', 'newspapers', 'root vegetables'), ('beef', 'newspapers', 'soda'), ('beef', 'newspapers', 'whole milk'), ('beef', 'newspapers', 'yogurt'), ('beef', 'other vegetables', 'root vegetables'), ('beef', 'other vegetables', 'soda'), ('beef', 'other vegetables', 'whole milk'), ('beef', 'other vegetables', 'yogurt'), ('beef', 'root vegetables', 'soda'), ('beef', 'root vegetables', 'whole milk'), ('beef', 'root vegetables', 'yogurt'), ('beef', 'soda', 'whole milk'), ('beef', 'soda', 'yogurt'), ('beef', 'whole milk', 'yogurt'), ('berries', 'other vegetables', 'rolls/buns')

In [28]:
a.L2.sort_values()[-20:]

(rolls/buns, tropical fruit)           91
(other vegetables, tropical fruit)     94
(shopping bags, whole milk)            95
(pastry, whole milk)                   97
(pip fruit, whole milk)                99
(citrus fruit, whole milk)            107
(bottled water, whole milk)           107
(bottled beer, whole milk)            107
(root vegetables, whole milk)         113
(rolls/buns, yogurt)                  117
(other vegetables, yogurt)            121
(rolls/buns, soda)                    121
(tropical fruit, whole milk)          123
(sausage, whole milk)                 134
(other vegetables, soda)              145
(other vegetables, rolls/buns)        158
(whole milk, yogurt)                  167
(soda, whole milk)                    174
(rolls/buns, whole milk)              209
(other vegetables, whole milk)        222
dtype: int64

### from the above result we observe that the products (citrus fruit, whole milk)            107
### (bottled water, whole milk)           107
### (bottled beer, whole milk)            107
### (root vegetables, whole milk)         113
### (rolls/buns, yogurt)                  117
### (other vegetables, yogurt)            121
### (rolls/buns, soda)                    121
### (tropical fruit, whole milk)          123
### (sausage, whole milk)                 134
### (other vegetables, soda)              145
### (other vegetables, rolls/buns)        158
### (whole milk, yogurt)                  167
### (soda, whole milk)                    174
### (rolls/buns, whole milk)              209
### (other vegetables, whole milk)  are bought together the most. So, the shop owner can concentrate on placing these products together or giving a combo offer to attract more customers. 